### post获取某公司的年报列表

In [3]:
import pandas as pd 
URL = 'http://www.cninfo.com.cn/new/hisAnnouncement/query'


MAX_PAGESIZE = 50
MAX_RELOAD_TIMES = 5
RESPONSE_TIMEOUT = 10

OUTPUT_FILENAME = 'report'
# 板块类型：沪市：shmb；深市：szse；深主板：szmb；中小板：szzx；创业板：szcy；
PLATE = 'szzx;'
# 公告类型：category_scgkfx_szsh（首次公开发行及上市）、category_ndbg_szsh（年度报告）、category_bndbg_szsh（半年度报告）
CATEGORY = 'category_ndbg_szsh;'


### 获取定制query部分

SZ_LIST = ['300','301','000','001','002','003']
SH_LIST = ['601','603','605','688','689','600']

code_df = pd.read_csv('../code_orgId.csv',dtype=str,)
code = code_df.set_index('code')

def get_s_query_dict(stock_code):
    stock_code = str(stock_code)
    orgId = code.loc[stock_code]['orgId']
    if stock_code[:3] in SZ_LIST:
        # sz创业
            res = {
                "stock":str(stock_code)+","+orgId,
                "column":"szse",
                'plate': 'sz',

            }
    elif stock_code[:3] in SH_LIST:
        # sh创业
            res = {
                "stock":str(stock_code)+","+orgId,
                "column":"sse",
                'plate': 'sh',

            }
    return res

START_DATE = '2013-01-01'
END_DATE = '2018-01-01'


page_num = 1
# 参数：页面id(每页条目个数由MAX_PAGESIZE控制)，是否返回总条目数(bool)
# def get_response(page_num,stock_code,return_total_count=False,):

def get_query_from_code(stock_code,):
    q_dict = get_s_query_dict(stock_code,)
    # print(q_dict)
    query_new = {
            'tabName': 'fulltext', 
            'pageSize': MAX_PAGESIZE,
            'pageNum': page_num,
            'category': CATEGORY,
            'seDate': START_DATE + '~' + END_DATE,
            'searchkey': '',
            'secid':'',
            'trade': '',
            'sortName': '',
            'sortType': '',
            'isHLtitle':'true'
            }
    q_dict.update(query_new)
    return q_dict

In [5]:
q = get_query_from_code('000006')
q

{'stock': '000006,gssz0000006',
 'column': 'szse',
 'plate': 'sz',
 'tabName': 'fulltext',
 'pageSize': 50,
 'pageNum': 1,
 'category': 'category_ndbg_szsh;',
 'seDate': '2013-01-01~2018-01-01',
 'searchkey': '',
 'secid': '',
 'trade': '',
 'sortName': '',
 'sortType': '',
 'isHLtitle': 'true'}

#### 使用asyncio访问

In [1]:
import asyncio,aiohttp

In [6]:
async with aiohttp.ClientSession() as session:
    res = await session.post(URL,data=q,proxy='http://127.0.0.1:7890')
    json = await res.json()

##### 对json的剖析

In [10]:
json

{'classifiedAnnouncements': None,
 'totalSecurities': 0,
 'totalAnnouncement': 11,
 'totalRecordNum': 11,
 'announcements': [{'id': None,
   'secCode': '000006',
   'secName': '深振业Ａ',
   'orgId': 'gssz0000006',
   'announcementId': '1203224890',
   'announcementTitle': '2016年年度报告',
   'announcementTime': 1490803200000,
   'adjunctUrl': 'finalpage/2017-03-30/1203224890.PDF',
   'adjunctSize': 3808,
   'adjunctType': 'PDF',
   'storageTime': None,
   'columnId': '09020202||250101||250102||251302',
   'pageColumn': 'SZZB',
   'announcementType': '01010503||010112||01030101',
   'associateAnnouncement': None,
   'important': None,
   'batchNum': None,
   'announcementContent': '',
   'orgName': None,
   'announcementTypeName': None},
  {'id': None,
   'secCode': '000006',
   'secName': '深振业Ａ',
   'orgId': 'gssz0000006',
   'announcementId': '1203224888',
   'announcementTitle': '2016年年度报告摘要',
   'announcementTime': 1490803200000,
   'adjunctUrl': 'finalpage/2017-03-30/1203224888.PDF',
   '

In [13]:
json['announcements'][-1]

{'id': None,
 'secCode': '000006',
 'secName': '深振业Ａ',
 'orgId': 'gssz0000006',
 'announcementId': '62301903',
 'announcementTitle': '2012年年度报告摘要',
 'announcementTime': 1364572800000,
 'adjunctUrl': 'finalpage/2013-03-30/62301903.PDF',
 'adjunctSize': 317,
 'adjunctType': 'PDF',
 'storageTime': None,
 'columnId': '250101||250102||251302',
 'pageColumn': 'SZZB',
 'announcementType': '01010501||010112||01030110',
 'associateAnnouncement': None,
 'important': None,
 'batchNum': None,
 'announcementContent': '',
 'orgName': None,
 'announcementTypeName': None}

In [12]:
len(json['announcements'])

11

### get获取搜索列表

#### 使用asyncio访问

In [ ]:
import asyncio,aiohttp

In [46]:
url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={}&sdate={}&edate={}&isfulltext={}'\
    '&sortName=stockcode_cat&sortType={}&pageNum={}&type={}'


url1 = url.format('企业社会责任','','','false','desc',2,'s')

In [47]:
url1

'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=企业社会责任&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=desc&pageNum=2&type=s'

In [48]:
async with aiohttp.ClientSession() as session:
    res = await session.get(url1,proxy='http://127.0.0.1:7890')
    json = await res.json()

##### 对json的剖析

In [39]:
json

{'classifiedAnnouncements': None,
 'totalSecurities': 0,
 'totalAnnouncement': 1409,
 'totalRecordNum': 1409,
 'announcements': [{'id': None,
   'secCode': '900956',
   'secName': '东贝Ｂ股',
   'orgId': 'gssh0900956',
   'announcementId': '63929535',
   'announcementTitle': '东贝Ｂ股：东贝B股2013年度<em>企业</em><em>社会</em><em>责任</em>报告',
   'announcementTime': 1398441605000,
   'adjunctUrl': 'finalpage/2014-04-26/63929535.PDF',
   'adjunctSize': 4951,
   'adjunctType': 'PDF',
   'storageTime': None,
   'columnId': '250401||251302',
   'pageColumn': 'SHZB',
   'announcementType': '01010501||010113||01239999',
   'associateAnnouncement': None,
   'important': None,
   'batchNum': None,
   'announcementContent': None,
   'orgName': None,
   'announcementTypeName': None},
  {'id': None,
   'secCode': '872138',
   'secName': '华博创科',
   'orgId': 'gfbj0872138',
   'announcementId': '1205523253',
   'announcementTitle': '[临时公告]华博创科:关于获得&ldquo;北京民营<em>企业</em><em>社会</em><em>责任</em>百强&rdquo; 荣誉称号的公告',
   'anno

##### json内部内容

In [40]:
json['announcements'][-1]

{'id': None,
 'secCode': '836208',
 'secName': '青矩技术',
 'orgId': 'gfbj0836208',
 'announcementId': '1204281600',
 'announcementTitle': '[临时公告]天职咨询:关于子公司入选2017 年度北京市非公有制<em>企业</em>履行<em>社会</em><em>责任</em>评价活动百家上榜单位名单的公告',
 'announcementTime': 1514217600000,
 'adjunctUrl': 'finalpage/2017-12-26/1204281600.PDF',
 'adjunctSize': 280,
 'adjunctType': 'PDF',
 'storageTime': None,
 'columnId': '2510',
 'pageColumn': 'GFBJ',
 'announcementType': '01010519',
 'associateAnnouncement': None,
 'important': None,
 'batchNum': None,
 'announcementContent': None,
 'orgName': None,
 'announcementTypeName': None}

In [34]:
json['totalAnnouncement']

17

In [35]:
json['totalRecordNum']


17

##### json长度

In [49]:
len(json['announcements'])

7

## 总函数

### 逻辑

- 编写协程爬取json（统一获取）
- 转成dataframe
- 进行统计


### 分函数

In [2]:
import sys
sys.path.append('..')

##### 【 json获取 】

In [172]:
# for i in range(10):
#     print(1)
#     if i > 2:
#         break
# else:
#     print(0)

1
1
1
1


In [4]:
import myRequests as requests
from asyncRequests import async_gets_jsons
def get_json_from_juchao(search_dict):

    for key,value in search_dict.items():
        if type(value) == list:
            multi_key = key
            values_list = value
            search_dict[multi_key] = f"""{{{multi_key}}}"""
            mode = "coroutine"
            break
    else:
        mode = "singal" 


    # 获取url
    url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={searchkey}'\
        '&sdate={sdate}'\
        '&edate={edate}'\
        '&isfulltext={isfulltext}'\
        '&sortName={sortName}'\
        '&sortType={sortType}'\
        '&pageNum={pageNum}&type={type}'.format(**search_dict)

    if mode == "singal":
        response = requests.get(url,)
        res = response.json()
    elif mode == "coroutine":
        urls = [url.format(**{multi_key:value}) for value in values_list]
        res = async_gets_jsons(urls)
    return res


def get_num_pages_from_json(json_dict):
    import math
    return math.ceil(json_dict['totalAnnouncement']/10)


###### 格式转换

In [90]:
import re
def remove_em(string):
    """
    去除搜索结果中的标记
    如将
    '香港中华煤气：环境、<em>社会</em><em>及</em><em>管治</em>报告2019'
    变为
    '香港中华煤气：环境、社会及管治报告2019'
    """
    string = string.replace('<em>','').replace('</em>','')
    return string


def to_realtime(timestamp):
    """return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(timestamp))"""
    import time
    time_local = time.localtime(timestamp)
    return time.strftime("%Y-%m-%d %H:%M:%S",time_local)

class myTime:
    def __init__(self,timestamp) -> None:
        import time
        time_local = time.localtime(timestamp)
        self.date_time = time.strftime("%Y-%m-%d %H:%M:%S",time_local)
        self.date = time.strftime("%Y-%m-%d",time_local)
        self.year = time.strftime("%Y",time_local)
        self.month = time.strftime("%m",time_local)
        self.day = time.strftime("%d",time_local)


### 主函数

In [5]:
# url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={}&sdate={}&edate={}&isfulltext={}'\
    # '&sortName=stockcode_cat&sortType={}&pageNum={}&type={}'


# url1 = url.format('企业社会责任','','','false','desc',2,'s')

if __name__=='__main__':
    param_dict = {
        # 搜索的关键词，如【'企业社会责任'】
        'searchkey':'社会及管治',
        # 发布起始时间，如【'2023-01-04'】【''表示不进行限制 】 
        'sdate':'', 
        # 发布截止时间，如【'2023-02-24'】【''表示不进行限制 】
        'edate':'', 
        # 是否进行全文搜索，【'false'为只搜索标题，'true'为搜索全文 】
        'isfulltext':'false', 
        # 按什么进行排序，【'pubdate'为按时间排序，'stockcode_cat'为按代码排序，'nothing'为按相关度排序 】
        'sortName':'stockcode_cat', 
        # 升序还是降序，【'asc'为升序，'desc'为降序 】
        'sortType':'asc', 
        # 页码，【如'1'】
        'pageNum':'1', 
        # 板块筛选，【''为全部板块，'shj'为深沪京，'s'为三板... 】
        'type':'', 
    }

    json_res = get_json_from_juchao(search_dict=param_dict)
    num_pages = get_num_pages_from_json(json_res)
    num_pages = 10
    param_dict['pageNum'] = [i+1 for i in range(num_pages)]
    get_json_from_juchao(search_dict=param_dict)

/tmp/ipykernel_538693/3880236858.py:31: RuntimeWarning: coroutine 'get_json_from_juchao.<locals>.async_gets_jsons' was never awaited
  get_json_from_juchao(search_dict=param_dict)


##### 测试format list

In [188]:
# url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={}&sdate={}&edate={}&isfulltext={}'\
    # '&sortName=stockcode_cat&sortType={}&pageNum={}&type={}'


# url1 = url.format('企业社会责任','','','false','desc',2,'s')

if __name__=='__main__':
    param_dict = {
        # 搜索的关键词，如【'企业社会责任'】
        'searchkey':'社会及管治',
        # 发布起始时间，如【'2023-01-04'】【''表示不进行限制 】 
        'sdate':'', 
        # 发布截止时间，如【'2023-02-24'】【''表示不进行限制 】
        'edate':'', 
        # 是否进行全文搜索，【'false'为只搜索标题，'true'为搜索全文 】
        'isfulltext':'false', 
        # 按什么进行排序，【'pubdate'为按时间排序，'stockcode_cat'为按代码排序，'nothing'为按相关度排序 】
        'sortName':'stockcode_cat', 
        # 升序还是降序，【'asc'为升序，'desc'为降序 】
        'sortType':'asc', 
        # 页码，【如'1'】
        'pageNum':['1','2','3','4'], 
        # 板块筛选，【''为全部板块，'shj'为深沪京，'s'为三板... 】
        'type':'', 
    }

    json_res = get_json_from_juchao(search_dict=param_dict)
    print(json_res)
    # num_pages = get_num_pages_from_json(json_res)
    # dicts = []
    # for i in num_pages:
    #     get_json_from_juchao(search_dicts=dicts)

['http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=1&type=', 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=2&type=', 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=3&type=', 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=4&type=']


##### 测试format

In [182]:
if 1:    
    param_dict = {
        # 搜索的关键词，如【'企业社会责任'】
        'searchkey':'社会及管治',
        # 发布起始时间，如【'2023-01-04'】【''表示不进行限制 】 
        'sdate':'', 
        # 发布截止时间，如【'2023-02-24'】【''表示不进行限制 】
        'edate':'', 
        # 是否进行全文搜索，【'false'为只搜索标题，'true'为搜索全文 】
        'isfulltext':'false', 
        # 按什么进行排序，【'pubdate'为按时间排序，'stockcode_cat'为按代码排序，'nothing'为按相关度排序 】
        'sortName':'stockcode_cat', 
        # 升序还是降序，【'asc'为升序，'desc'为降序 】
        'sortType':'asc', 
        # 页码，【如'1'】
        'pageNum':'{pageNum}', 
        # 板块筛选，【''为全部板块，'shj'为深沪京，'s'为三板... 】
        'type':'', 
    }
    url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={searchkey}'\
        '&sdate={sdate}'\
        '&edate={edate}'\
        '&isfulltext={isfulltext}'\
        '&sortName={sortName}'\
        '&sortType={sortType}'\
        '&pageNum={pageNum}&type={type}'.format(**param_dict)
    print(url)


http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum={pageNum}&type=


In [184]:
url.format(**{'pageNum':1})

'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=1&type='

In [127]:
num_pages

577

In [181]:
param_dict.__format__({'searchkey':1})
format(param_dict,)

TypeError: __format__() argument must be str, not dict

##### 测试json的get

#### 字典与dataframe

In [17]:
param_dict1 = {
        # 搜索的关键词，如【'企业社会责任'】
        'searchkey':'社会及管治',
        # 发布起始时间，如【'2023-01-04'】【''表示不进行限制 】 
        'sdate':'', 
        # 发布截止时间，如【'2023-02-24'】【''表示不进行限制 】
        'edate':'', 
        # 是否进行全文搜索，【'false'为只搜索标题，'true'为搜索全文 】
        'isfulltext':'false', 
        # 按什么进行排序，【'pubdate'为按时间排序，'stockcode_cat'为按代码排序，'nothing'为按相关度排序 】
        'sortName':'stockcode_cat', 
        # 升序还是降序，【'asc'为升序，'desc'为降序 】
        'sortType':'asc', 
        # 页码，【如'1'】
        'pageNum':'1', 
        # 板块筛选，【''为全部板块，'shj'为深沪京，'s'为三板... 】
        'type':'', 
    }
param_dict2 = {
        # 搜索的关键词，如【'企业社会责任'】
        'searchkey':'社会及管治',
        # 发布起始时间，如【'2023-01-04'】【''表示不进行限制 】 
        'sdate':'', 
        # 发布截止时间，如【'2023-02-24'】【''表示不进行限制 】
        'edate':'', 
        # 是否进行全文搜索，【'false'为只搜索标题，'true'为搜索全文 】
        'isfulltext':'false', 
        # 按什么进行排序，【'pubdate'为按时间排序，'stockcode_cat'为按代码排序，'nothing'为按相关度排序 】
        'sortName':'stockcode_cat', 
        # 升序还是降序，【'asc'为升序，'desc'为降序 】
        'sortType':'asc', 
        # 页码，【如'1'】
        'pageNum':'1', 
        # 板块筛选，【''为全部板块，'shj'为深沪京，'s'为三板... 】
        'type':'', 
    }


In [26]:
s1 = pd.Series(param_dict1)

In [27]:
s1.to_frame()

,0
searchkey,社会及管治
sdate,
edate,
isfulltext,false
sortName,stockcode_cat
sortType,asc
pageNum,1
type,


In [23]:
[i for i in zip(param_dict1,param_dict2)]
param_dict1. param_dict2

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [60]:
{key:[value] for key,value in param_dict1.items()}['pageNum'] = ['1','2']
param_dict1

{'searchkey': '社会及管治',
 'sdate': '',
 'edate': '',
 'isfulltext': 'false',
 'sortName': 'stockcode_cat',
 'sortType': 'asc',
 'pageNum': '1',
 'type': ''}

In [61]:
{key:[value] for key,value in param_dict1.items()}

{'searchkey': ['社会及管治'],
 'sdate': [''],
 'edate': [''],
 'isfulltext': ['false'],
 'sortName': ['stockcode_cat'],
 'sortType': ['asc'],
 'pageNum': ['1'],
 'type': ['']}

In [37]:
df = pd.DataFrame(columns=param_dict1.keys())
df

,searchkey,sdate,edate,isfulltext,sortName,sortType,pageNum,type


In [39]:
pd.DataFrame(param_dict1,ignore_index=True)


TypeError: __init__() got an unexpected keyword argument 'ignore_index'

In [40]:
pd.DataFrame({key:[value] for key,value in param_dict1.items()})#,ignore_index=True)
# df.concat(param_dict1,ignore_index=True)


,searchkey,sdate,edate,isfulltext,sortName,sortType,pageNum,type
0,社会及管治,,,false,stockcode_cat,asc,1,


In [41]:
pd.concat()


,searchkey,sdate,edate,isfulltext,sortName,sortType,pageNum,type


In [44]:
df2 = pd.DataFrame([['c', 3], ['d', 4],['e','']],columns=['letter', 'number'])
df2

,letter,number
0,c,3
1,d,4
2,e,


##### 这里是announcements_list字典列表

In [52]:
announcements_list = [{'id': None, 'secCode': '000039', 'secName': '中集集团', 'orgId': 'gssz0000039', 'announcementId': '1204526484', 'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1522166400000, 'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF', 'adjunctSize': 2190, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000039', 'secName': '中集集团', 'orgId': 'gssz0000039', 'announcementId': '1205949450', 'announcementTitle': '中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1553702400000, 'adjunctUrl': 'finalpage/2019-03-28/1205949450.PDF', 'adjunctSize': 21279, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000039', 'secName': '中集集团', 'orgId': 'gssz0000039', 'announcementId': '1203671603', 'announcementTitle': '中集集团：2016年环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1498838400000, 'adjunctUrl': 'finalpage/2017-07-01/1203671603.PDF', 'adjunctSize': 1975, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010501||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000039', 'secName': '中集集团', 'orgId': 'gssz0000039', 'announcementId': '1207411179', 'announcementTitle': '中集集团：<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告（2019）', 'announcementTime': 1585238400000, 'adjunctUrl': 'finalpage/2020-03-27/1207411179.PDF', 'adjunctSize': 18857, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000166', 'secName': '申万宏源', 'orgId': 'qsgn0000301', 'announcementId': '1206470370', 'announcementTitle': '申万宏源：2018年度环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1563897600000, 'adjunctUrl': 'finalpage/2019-07-24/1206470370.PDF', 'adjunctSize': 6094, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000507', 'secName': '珠海港', 'orgId': 'gssz0000507', 'announcementId': '1209714074', 'announcementTitle': '珠海港：珠海港股份有限公司2020年度环境、<em>社会</em><em>及</em><em>管治</em>（ESG）报告', 'announcementTime': 1618588800000, 'adjunctUrl': 'finalpage/2021-04-17/1209714074.PDF', 'adjunctSize': 17482, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000507', 'secName': '珠海港', 'orgId': 'gssz0000507', 'announcementId': '1213142096', 'announcementTitle': '珠海港：2021年度环境、<em>社会</em><em>及</em><em>管治</em>（ESG）报告', 'announcementTime': 1650988800000, 'adjunctUrl': 'finalpage/2022-04-27/1213142096.PDF', 'adjunctSize': 37221, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010501||010112||012399', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000513', 'secName': '丽珠集团', 'orgId': 'gssz0000513', 'announcementId': '1210132133', 'announcementTitle': '丽珠集团：2020年度环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1622476800000, 'adjunctUrl': 'finalpage/2021-06-01/1210132133.PDF', 'adjunctSize': 11675, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000513', 'secName': '丽珠集团', 'orgId': 'gssz0000513', 'announcementId': '1213517593', 'announcementTitle': '丽珠集团：2021年度环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1653580800000, 'adjunctUrl': 'finalpage/2022-05-27/1213517593.PDF', 'adjunctSize': 37828, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010501||010112||012399', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}, {'id': None, 'secCode': '000513', 'secName': '丽珠集团', 'orgId': 'gssz0000513', 'announcementId': '1205053842', 'announcementTitle': '丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1528732800000, 'adjunctUrl': 'finalpage/2018-06-12/1205053842.PDF', 'adjunctSize': 6408, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010501||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}]

In [146]:
print(announcements_list[0])

{'id': None, 'secCode': '000039', 'secName': '中集集团', 'orgId': 'gssz0000039', 'announcementId': '1204526484', 'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告', 'announcementTime': 1522166400000, 'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF', 'adjunctSize': 2190, 'adjunctType': 'PDF', 'storageTime': None, 'columnId': '09020202||250101||251302', 'pageColumn': 'SZZB', 'announcementType': '01010503||010112||01239999', 'associateAnnouncement': None, 'important': None, 'batchNum': None, 'announcementContent': None, 'orgName': None, 'announcementTypeName': None}


In [147]:
'a'.replace('a','1')

'1'

##### 这里是pdfs的names_urls

In [148]:
pdf_urls = [('金山办公：金山办公2021年度环境、社会及管治（ESG）报告（英文版）.pdf', 'http://static.cninfo.com.cn/finalpage/2022-04-07/1212824161.PDF'), ('路德环境：2021年度环境、社会及管治（ESG）报告.pdf', 'http://static.cninfo.com.cn/finalpage/2022-04-21/1213004008.PDF'), ('南网科技：南方电网电力科技股份有限公司2021年度环境、社会及管治（ESG）报告.pdf', 'http://static.cninfo.com.cn/finalpage/2022-05-31/1213550938.PDF'), ('复旦微电：上海复旦微电子集团股份有限公司董事会环境、社会及管治委员会工作细则.pdf', 'http://static.cninfo.com.cn/finalpage/2021-12-07/1211822593.PDF'), ('复旦张江：上海复旦张江生物医药股份有限公司环境、社会及管治报告.pdf', 'http://static.cninfo.com.cn/finalpage/2021-03-26/1209454399.PDF'), ('天合光能：天合光能股份有限公司2021年度环境、社会及管治（ESG）报告.pdf', 'http://static.cninfo.com.cn/finalpage/2022-04-28/1213183962.PDF'), ('金盘科技：2021年环境、社会及管治（ESG）报告.pdf', 'http://static.cninfo.com.cn/finalpage/2022-04-16/1212944843.PDF'), ('中芯国际：中芯国际2021年环境社会及管治（ESG）报告-提升版.pdf', 'http://static.cninfo.com.cn/finalpage/2022-06-01/1213571733.PDF'), ('中芯国际：中芯国际2021年环境、社会及管治（ESG）报告.pdf', 'http://static.cninfo.com.cn/finalpage/2022-03-31/1212753877.PDF')]

##### 这里是pdfs的dirs

In [151]:
dirs = ('./data/search_res/中集集团：2017年社会责任暨环境、社会及管治报告.pdf', './data/search_res/中集集团：2018社会责任暨环境、社会及管治报告.pdf', './data/search_res/中集集团：社会责任暨环境、社会及管治报告（2019）.pdf', './data/search_res/中集集团：2016年环境、社会及管治报告.pdf', './data/search_res/申万宏源：2018年度环境、社会及管治报告.pdf', './data/search_res/珠海港：珠海港股份有限公司2020年度环境、社会及管治（ESG）报告.pdf', './data/search_res/珠海港：2021年度环境、社会及管治（ESG）报告.pdf', './data/search_res/丽珠集团：2020年度环境、社会及管治报告.pdf', './data/search_res/丽珠集团：2021年度环境、社会及管治报告.pdf', './data/search_res/丽珠集团：2017年度环境、社会及管治报告.pdf')

In [152]:
len(dirs)

10

##### 一个parse pdf的函数

In [159]:
pdf_dirs = ['.'+dir for dir in dirs]
pdf_dirs

['../data/search_res/中集集团：2017年社会责任暨环境、社会及管治报告.pdf',
 '../data/search_res/中集集团：2018社会责任暨环境、社会及管治报告.pdf',
 '../data/search_res/中集集团：社会责任暨环境、社会及管治报告（2019）.pdf',
 '../data/search_res/中集集团：2016年环境、社会及管治报告.pdf',
 '../data/search_res/申万宏源：2018年度环境、社会及管治报告.pdf',
 '../data/search_res/珠海港：珠海港股份有限公司2020年度环境、社会及管治（ESG）报告.pdf',
 '../data/search_res/珠海港：2021年度环境、社会及管治（ESG）报告.pdf',
 '../data/search_res/丽珠集团：2020年度环境、社会及管治报告.pdf',
 '../data/search_res/丽珠集团：2021年度环境、社会及管治报告.pdf',
 '../data/search_res/丽珠集团：2017年度环境、社会及管治报告.pdf']

In [225]:
import pdfplumber
# def parse_pdf(pdf_dir):
pdf_dir = pdf_dirs[0]
# pdf_dir = '../data/search_res/中集集团：2017年社会责任暨环境、社会及管治报告.pdf'
if 1:
    res = {
        'pages_num':0,
        'words_num':0,
    }
    pp = pdfplumber.open(pdf_dir)
    res['pages_num'] = len(pp.pages)
    # return res

class parsePdf:
    def __init__(self,pdf_dir) -> None:
        self.pdf = pdfplumber.open(pdf_dir)
        self.pages = self.pdf.pages


    def get_pages_num(self):
        return len(self.pages)


    def get_words_num(self,):#ignore_text_list=[]):
        # ignore_text_list += '\n'
        counts = len(self.pdf.chars)
        # for page in self.pages:
        #     page.extract_text
        #     counts += 
        return counts


    def __del__(self):
        self.pdf.close()
        # print('文件已关闭')
        return





In [283]:
pdf_dirs[6]

'../data/search_res/珠海港：2021年度环境、社会及管治（ESG）报告.pdf'

In [280]:
pdf = parsePdf(pdf_dirs[6])
print(pdf.get_pages_num(),pdf.get_words_num())

56 59388


In [292]:
po = pdfplumber.open(pdf_dirs[6])

In [293]:
pages =  po.pages

In [294]:
a = pages[0].chars

In [295]:
for page in pages:
    print(page.page_number)
    print(len(page.chars))    

1
0
2
40
3
1062
4
1177
5
1269
6
1085
7
1219
8
642
9
1075
10
926
11
78
12
1530
13
1012
14
1409
15
1257
16
827
17
269
18
1247
19
1248
20
1407
21
1468
22
371
23
2411
24
197
25
1362
26
1430
27
1446
28
1321
29
1162
30
1454
31
1344
32
926
33
762
34
269
35
1052
36
1338
37
1202
38
935
39
476
40
1128
41
1102
42
1837
43
1524
44
1573
45
1969
46
139
47
1399
48
1063
49
1182
50
139
51
1060
52
1015
53
894
54
1949
55
1710
56
0


In [290]:
''.join([char['text'] for char in po.pages[7].chars])

'02121112走进珠海港珠海港股份有限公司 2021年度环境、社会及管治报告     在本年度ESG报告的准备过程中，我们邀请了专业顾问对本公司年度可持续发展议题进行了回顾及评估，最终归纳、更新及总结了本年度对珠海港具有实质性的ESG议题，作为本报告的编制基础。议题评估结果将为未来珠海港ESG管理工作提供重要参考。珠海港2021年ESG议题重大性分析矩阵对企业的重要性对利益相关方的重要性高低高步骤一：议题库审阅与更新                审阅2020年的重要性议题评估结果，结合2021年公司整体业务发展情况，并对标同行企业ESG管理实践，综合考量并                调整更新，形成2021年珠海港ESG议题库。步骤二：利益相关方沟通与重要性议题分析                在往年利益相关方沟通的基础上，结合公司自身业务经营情况，对各项议题进行评估与排序，形成议题矩阵。步骤三：重要性议题回应及披露                根据议题的重要性，在报告中进行重点回应与披露。根据以上步骤，最终得出的重大性议题矩阵如下：公益慈善气候变化风险及应对区域经济发展与建设负责任供应链管理多方合作与行业发展员工关怀保障稳定供气供电清洁能源与产品绿色机遇生态与生物多样性排放物管理员工培训与发展水资源和废水管理响应国家政策绿色智慧港口建设信息安全与客户隐私保护客户服务与产品质量员工权益保障守法经营与风险控制廉洁建设企业管治与合规披露安全生产重要性议题评估'

In [282]:
len(po.chars)

59388

In [291]:
po.close()

##### 使用parse_pdf

In [277]:
from parsePdf import parse_pdf

In [279]:
parse_pdf(pdf_dirs[6])

../data/search_res/珠海港：2021年度环境、社会及管治（ESG）报告.pdf {'pages_num': 56, 'words_num': 58794}


{'pages_num': 56, 'words_num': 58794}

In [ ]:
cont = 1
for page in pp.pages:
    cont += len(page.chars)
cont

In [222]:
parsePdf(pdf_dir).get_pages_num()

81

In [226]:
parsePdf(pdf_dir).get_words_num()


43728

In [192]:
page0 = pp.pages[4]

In [201]:
ext = page0.extract_words()
print(ext)

[{'text': '关于本报告', 'x0': 246.0138, 'x1': 335.1336, 'top': 73.38589999999999, 'doctop': 3304.8819000000003, 'bottom': 91.38589999999999, 'upright': True, 'direction': 1}, {'text': '04', 'x0': 285.84, 'x1': 295.256, 'top': 114.20979999999997, 'doctop': 3345.7058, 'bottom': 122.20979999999997, 'upright': True, 'direction': 1}, {'text': '发布形式', 'x0': 56.6929, 'x1': 108.69290000000001, 'top': 155.81459999999993, 'doctop': 3387.3106, 'bottom': 168.81459999999993, 'upright': True, 'direction': 1}, {'text': '报告反馈', 'x0': 297.6378, 'x1': 349.6378, 'top': 155.81459999999993, 'doctop': 3387.3106, 'bottom': 168.81459999999993, 'upright': True, 'direction': 1}, {'text': '本报告分别以中、英文版本通过网络发布。', 'x0': 56.6929, 'x1': 283.4929000000001, 'top': 184.5145, 'doctop': 3416.0105000000003, 'bottom': 194.5145, 'upright': True, 'direction': 1}, {'text': '于报告编写过程中，我们力求报告内容平实、清', 'x0': 297.6378, 'x1': 524.3768, 'top': 184.5145, 'doctop': 3416.0105000000003, 'bottom': 194.5145, 'upright': True, 'direction': 1}, {'t

In [217]:
len(pp.chars)

43728

In [216]:
cont = 1
for page in pp.pages:
    cont += len(page.chars)
cont

43729

In [239]:
txt = pp.pages[3].extract_text()

In [241]:
len(txt.replace('\n',''))

698

In [244]:
len(txt)

719

In [245]:
a = " 这是个好日子"
len(a)

7

In [246]:
ignore_text_list=[]
ignore_text_list += '\n'
ignore_text_list

['\n']

In [236]:
len(pp.pages[3].chars)

688

In [238]:
for i in pp.pages[3].chars:
    print(i['text'],end='')

关于本报告032017年度社会责任暨环境、社会及管治报告本报告为中国国际海运集装箱（集团）股份有限公司（“本公司”与其子公司合称“本集团”）自2008年发布《社会责任报告》及2017年发布《环境、社会及管治报告》以来，首次将两者有机融合而形成的第一份《社会责任暨环境、社会及管治报告》。本报告客观反映本集团在追求自身发展的同时，积极践行社会责任，为社会及企业自身可持续发展所付出的努力及所取得的成效。本公司董事会清楚确认报告真实性的责任，对本集团的环境及社会管治策略及汇报承担全部责任，并已审阅及批准本报告。报告期及范围本报告与本公司《2017年年报》相辅相成，披露本公司从2017年1月1日起至2017年12月31日止（“年内”部分内容延伸至2017年以前），于履行社会责任方面（包括环境、社会和管治）的管理方法及部分考量面的表现，涵盖本公司各产业板块及成员企业。有关企业管治的进一步内容，请参阅公司《2017年年报》“公司治理及企业管治报告”章节。针对本公司能源、化工及液态食品装备业务的环境、社会及管治方面的详细内容，请参见将于2017年6月底发布的中集安瑞科控股有限公司（上市编号：3899）的《2017年环境、社会及管治报告》。报告主要参考标准本报告以《中国工业企业及工业协会社会责任指南（第二版）》为框架，遵守《深圳证券交易所上市公司社会责任指引》、《香港联合交易所主板上市规则》附录二十七《环境、社会及管治报告指引》（“ESG报告指引”）及《全球报告倡议组织可持续发展报告标准》“核心”符合选项进行编制。发布周期本报告每年发布一次，具体发布时间为每年3月或4月。

In [200]:
page0.extract_text()

'关于本报告\n04\n发布形式 报告反馈\n本报告分别以中、英文版本通过网络发布。 于报告编写过程中，我们力求报告内容平实、清\n股 东 及 各 利 益 相 关 方 可 登 陆 巨 潮 资 讯 网  晰、易懂。本报告内，我们遵照香港联合交易所\n(www.cninfo.com.cn) 和香港联交所披露易网站  《环境、社会及管治报告指引》首次披露环境类的\n(www.hkexnews.hk) 查阅本报告，如中文版与英文 关键绩效指标，本公司已建立相应工作体系尽可能\n译本存在歧义，概以中文版本为准。 保证所披露数据真实、准确。但受各种客观因素所\n限，我们在首次收集统计数据的过程上，可能存在\n内容备注 不完善的地方，报告内容也未必令利益相关方完全\n满意。本公司将继续努力完善报告，提高公司可持\n为便于表述，本报告分别使用“中集集团”、“中 续发展的披露水平。我们欢迎您对本报告提出意\n集”、“我们”等称谓代替“中国国际海运集装箱（集 见，以及对于报告内容的建议。请通过以下方式与\n团）股份有限公司”；用“中集集装箱”、“中集车 我们联系：\n辆”、“中集海工”、“中集安瑞科”、“中集金融”等\n称谓代替本公司旗下相应产业板块。 中国国际海运集装箱（集团）股份有限公司\n地址：中国广东省深圳市蛇口工业区港湾大道2号中\n集集团研发中心\n电话：86-755-26691130\n传真：86-755-26692707\n电邮：cimcgroup@cimc.com\n中国国际海运集装箱（集团）股份有限公司'

In [195]:
len(ext)

640

In [189]:

parse_res = [parse_pdf(dir) for dir in pdf_dirs]
pages_nums = [res['pages_num'] for res in parse_res]
pages_nums


[81, 100, 92, 32, 54, 41, 56, 138, 246, 70]

In [190]:

parse_res = [parsePdf(dir).get_pages_num() for dir in pdf_dirs]
pages_nums = [res for res in parse_res]
pages_nums


[81, 100, 92, 32, 54, 41, 56, 138, 246, 70]

##### 转化为页数列表和字数列表


In [229]:
parse_res = [{'pages_num': 81, 'words_num': 43728}, {'pages_num': 100, 'words_num': 63879}, {'pages_num': 92, 'words_num': 61393}, {'pages_num': 32, 'words_num': 19953}, {'pages_num': 54, 'words_num': 24963}, {'pages_num': 41, 'words_num': 37638}, {'pages_num': 56, 'words_num': 59388}, {'pages_num': 138, 'words_num': 75439}, {'pages_num': 246, 'words_num': 133257}, {'pages_num': 70, 'words_num': 36775}]

In [230]:
pages_nums = [res['pages_num'] for res in parse_res]
words_nums = [res['words_num'] for res in parse_res]

In [232]:
pages_nums

[81, 100, 92, 32, 54, 41, 56, 138, 246, 70]

In [233]:
words_nums

[43728, 63879, 61393, 19953, 24963, 37638, 59388, 75439, 133257, 36775]

##### 把页数写到dataframe中

In [168]:
pages_nums
df['pages_num'] = pages_nums
df

,secCode,secName,orgId,announcementId,announcementTitle,announcement_title,announcementTime,announcement_year,adjunctUrl,record_time,pages_num
0,000039,中集集团,gssz0000039,1204526484,中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</e...,中集集团：2017年社会责任暨环境、社会及管治报告,1522166400000,2018,finalpage/2018-03-28/1204526484.PDF,2023-01-17,81
1,000039,中集集团,gssz0000039,1205949450,中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em...,中集集团：2018社会责任暨环境、社会及管治报告,1553702400000,2019,finalpage/2019-03-28/1205949450.PDF,2023-01-17,100
2,000039,中集集团,gssz0000039,1203671603,中集集团：2016年环境、<em>社会</em><em>及</em><em>管治</em>报告,中集集团：2016年环境、社会及管治报告,1498838400000,2017,finalpage/2017-07-01/1203671603.PDF,2023-01-17,92
3,000039,中集集团,gssz0000039,1207411179,中集集团：<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em...,中集集团：社会责任暨环境、社会及管治报告（2019）,1585238400000,2020,finalpage/2020-03-27/1207411179.PDF,2023-01-17,32
4,000166,申万宏源,qsgn0000301,1206470370,申万宏源：2018年度环境、<em>社会</em><em>及</em><em>管治</em>报告,申万宏源：2018年度环境、社会及管治报告,1563897600000,2019,finalpage/2019-07-24/1206470370.PDF,2023-01-17,54
5,000507,珠海港,gssz0000507,1209714074,珠海港：珠海港股份有限公司2020年度环境、<em>社会</em><em>及</em><em...,珠海港：珠海港股份有限公司2020年度环境、社会及管治（ESG）报告,1618588800000,2021,finalpage/2021-04-17/1209714074.PDF,2023-01-17,41
6,000507,珠海港,gssz0000507,1213142096,珠海港：2021年度环境、<em>社会</em><em>及</em><em>管治</em>（...,珠海港：2021年度环境、社会及管治（ESG）报告,1650988800000,2022,finalpage/2022-04-27/1213142096.PDF,2023-01-17,56
7,000513,丽珠集团,gssz0000513,1210132133,丽珠集团：2020年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2020年度环境、社会及管治报告,1622476800000,2021,finalpage/2021-06-01/1210132133.PDF,2023-01-17,138
8,000513,丽珠集团,gssz0000513,1213517593,丽珠集团：2021年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2021年度环境、社会及管治报告,1653580800000,2022,finalpage/2022-05-27/1213517593.PDF,2023-01-17,246
9,000513,丽珠集团,gssz0000513,1205053842,丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2017年度环境、社会及管治报告,1528732800000,2018,finalpage/2018-06-12/1205053842.PDF,2023-01-17,70


##### 将字数写入

##### 时间相关

In [66]:
import time
timestamp = 1522166400#000
time_local = time.localtime(timestamp)
time.strftime("%Y-%m-%d %H:%M:%S",time_local)

'2018-03-28 00:00:10'

In [63]:
time.strftime("%Y",time_local)

'2018'

In [69]:
'2018-03-28 00:00:10'.split(' ')[0].split('-')

['2018', '03', '28']

In [86]:
to_realtime(time.time())

'2023-01-17 09:39:04'

In [88]:
to_realtime(1673919562.556)

'2023-01-17 09:39:22'

In [98]:
ITEM_REALTIME = 'announcement_realtime'
RECORD_TIME = 'record_time'

FIX_ROW_DICT = {
    'secCode':'',
    'secName':'',
    'orgId':'',
    'announcementTitle':'',
    'announcementTime':'',
    f'{ITEM_REALTIME}':'',# 实际时间如'2018-03-28 00:00:10'
    'adjunctUrl':'',
    f'{RECORD_TIME}':'' # 记录日期
}


def reformat_dict(announcement_dict):
    maintain_keys = ['secCode','secName','orgId','announcementTitle','announcementTime','adjunctUrl',]
    res_dict = {key:value for key,value in FIX_ROW_DICT.items()}
    res_dict.update({key:value for key,value in announcement_dict.items() if key in FIX_ROW_DICT.keys()})

    res_dict[ITEM_REALTIME] = myTime(announcement_dict['announcementTime']/1000).date_time
    res_dict[RECORD_TIME] = myTime(time.time()).date
    # return {key:value for key,value in announcement_dict.items() if key in maintain_keys}
    return res_dict

In [112]:

def reformat_dict(an_dict):
    res_dict = {
        'secCode':an_dict['secCode'],
        'secName':an_dict['secName'],
        'orgId':an_dict['orgId'],
        'announcementTitle':an_dict['announcementTitle'],
        'announcementTime':an_dict['announcementTime'],
        'announcement_realtime':myTime(an_dict['announcementTime']/1000).date_time, # 实际时间如'2018-03-28 00:00:10'
        'adjunctUrl':an_dict['adjunctUrl'],
        'record_time':myTime(time.time()).date, # 记录日期
    }
    return res_dict

In [113]:
d0 = reformat_dict(announcements_list[-1])
d0['test'][0] = 2
d0

{'code': '000513',
 'name': '丽珠集团',
 'orgId': 'gssz0000513',
 'announcementTitle': '丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告',
 'announcementTime': 1528732800000,
 'announcement_realtime': '2018-06-12 00:00:00',
 'adjunctUrl': 'finalpage/2018-06-12/1205053842.PDF',
 'record_time': '2023-01-17',
 'test': [2]}

In [115]:
reformat_dict(announcements_list[-1])

{'code': '000513',
 'name': '丽珠集团',
 'orgId': 'gssz0000513',
 'announcementTitle': '丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告',
 'announcementTime': 1528732800000,
 'announcement_realtime': '2018-06-12 00:00:00',
 'adjunctUrl': 'finalpage/2018-06-12/1205053842.PDF',
 'record_time': '2023-01-17',
 'test': [1]}

In [116]:
d1 = reformat_dict(announcements_list[-1])
list(d1.values())

TypeError: reformat_dict() got an unexpected keyword argument 'announcement_dict'

In [117]:
d1 = reformat_dict(announcements_list[-1])
list(d1.values())
res_dicts = [reformat_dict(announcement) for announcement in announcements_list]
ress_dicts = [list(res_dict.values()) for res_dict in res_dicts]

In [123]:
d2 = []
d2+= [['1']]
d2

[['1']]

In [120]:
pd.DataFrame(ress_dicts,columns=list(res_dicts[0].keys()))

,code,name,orgId,announcementTitle,announcementTime,announcement_realtime,adjunctUrl,record_time,test
0,000039,中集集团,gssz0000039,中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</e...,1522166400000,2018-03-28 00:00:00,finalpage/2018-03-28/1204526484.PDF,2023-01-17,[1]
1,000039,中集集团,gssz0000039,中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em...,1553702400000,2019-03-28 00:00:00,finalpage/2019-03-28/1205949450.PDF,2023-01-17,[1]
2,000039,中集集团,gssz0000039,中集集团：2016年环境、<em>社会</em><em>及</em><em>管治</em>报告,1498838400000,2017-07-01 00:00:00,finalpage/2017-07-01/1203671603.PDF,2023-01-17,[1]
3,000039,中集集团,gssz0000039,中集集团：<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em...,1585238400000,2020-03-27 00:00:00,finalpage/2020-03-27/1207411179.PDF,2023-01-17,[1]
4,000166,申万宏源,qsgn0000301,申万宏源：2018年度环境、<em>社会</em><em>及</em><em>管治</em>报告,1563897600000,2019-07-24 00:00:00,finalpage/2019-07-24/1206470370.PDF,2023-01-17,[1]
5,000507,珠海港,gssz0000507,珠海港：珠海港股份有限公司2020年度环境、<em>社会</em><em>及</em><em...,1618588800000,2021-04-17 00:00:00,finalpage/2021-04-17/1209714074.PDF,2023-01-17,[1]
6,000507,珠海港,gssz0000507,珠海港：2021年度环境、<em>社会</em><em>及</em><em>管治</em>（...,1650988800000,2022-04-27 00:00:00,finalpage/2022-04-27/1213142096.PDF,2023-01-17,[1]
7,000513,丽珠集团,gssz0000513,丽珠集团：2020年度环境、<em>社会</em><em>及</em><em>管治</em>报告,1622476800000,2021-06-01 00:00:00,finalpage/2021-06-01/1210132133.PDF,2023-01-17,[1]
8,000513,丽珠集团,gssz0000513,丽珠集团：2021年度环境、<em>社会</em><em>及</em><em>管治</em>报告,1653580800000,2022-05-27 00:00:00,finalpage/2022-05-27/1213517593.PDF,2023-01-17,[1]
9,000513,丽珠集团,gssz0000513,丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告,1528732800000,2018-06-12 00:00:00,finalpage/2018-06-12/1205053842.PDF,2023-01-17,[1]


{'secCode': '000513',
 'secName': '丽珠集团',
 'orgId': 'gssz0000513',
 'announcementTitle': '丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告',
 'announcementTime': 1528732800000,
 'announcement_realtime': '2018-06-12 00:00:00',
 'adjunctUrl': 'finalpage/2018-06-12/1205053842.PDF'}

In [51]:
import time 
import datetime
datetime.datetime.

datetime.datetime

In [13]:
param_dict.fromkeys(['type','pageNum'])

{'type': None, 'pageNum': None}

In [16]:
import pandas as pd
pd.DataFrame.from_dict(param_dict)

ValueError: If using all scalar values, you must pass an index

#### 1

In [132]:
param_dict

{'searchkey': '社会及管治',
 'sdate': '',
 'edate': '',
 'isfulltext': 'false',
 'sortName': 'stockcode_cat',
 'sortType': 'asc',
 'pageNum': ['1', '2'],
 'type': ''}

In [131]:
param_dict['pageNum'] = ['1','2']

In [133]:
type(param_dict['pageNum'])
param_dict.values

list

In [170]:
# if list in (type(value) for value in param_dict.values()):
#     print(1)
# if list in [type(value) for _,value in param_dict.items()]:
#     print(1)
# for _,value in param_dict.items():
#     if type(value) == list:
#         print(1)

1


1


#### dataframe操作

In [247]:
import pandas as pd
df = pd.read_csv('../data/SearchRes.csv',dtype=str)

In [260]:
df

,secCode,secName,orgId,announcementId,announcementTitle,announcement_title,announcementTime,announcement_year,adjunctUrl,record_time,pages_num,words_num
0,000039,中集集团,gssz0000039,1204526484,中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</e...,中集集团：2017年社会责任暨环境、社会及管治报告,1522166400000,2018,finalpage/2018-03-28/1204526484.PDF,2023-01-17,81,44213
1,000039,中集集团,gssz0000039,1205949450,中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em...,中集集团：2018社会责任暨环境、社会及管治报告,1553702400000,2019,finalpage/2019-03-28/1205949450.PDF,2023-01-17,100,73065
2,000039,中集集团,gssz0000039,1207411179,中集集团：<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em...,中集集团：社会责任暨环境、社会及管治报告（2019）,1585238400000,2020,finalpage/2020-03-27/1207411179.PDF,2023-01-17,92,63008
3,000039,中集集团,gssz0000039,1203671603,中集集团：2016年环境、<em>社会</em><em>及</em><em>管治</em>报告,中集集团：2016年环境、社会及管治报告,1498838400000,2017,finalpage/2017-07-01/1203671603.PDF,2023-01-17,32,19858
4,000166,申万宏源,qsgn0000301,1206470370,申万宏源：2018年度环境、<em>社会</em><em>及</em><em>管治</em>报告,申万宏源：2018年度环境、社会及管治报告,1563897600000,2019,finalpage/2019-07-24/1206470370.PDF,2023-01-17,54,24933
5,000507,珠海港,gssz0000507,1209714074,珠海港：珠海港股份有限公司2020年度环境、<em>社会</em><em>及</em><em...,珠海港：珠海港股份有限公司2020年度环境、社会及管治（ESG）报告,1618588800000,2021,finalpage/2021-04-17/1209714074.PDF,2023-01-17,41,37208
6,000507,珠海港,gssz0000507,1213142096,珠海港：2021年度环境、<em>社会</em><em>及</em><em>管治</em>（...,珠海港：2021年度环境、社会及管治（ESG）报告,1650988800000,2022,finalpage/2022-04-27/1213142096.PDF,2023-01-17,56,58794
7,000513,丽珠集团,gssz0000513,1210132133,丽珠集团：2020年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2020年度环境、社会及管治报告,1622476800000,2021,finalpage/2021-06-01/1210132133.PDF,2023-01-17,138,75402
8,000513,丽珠集团,gssz0000513,1213517593,丽珠集团：2021年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2021年度环境、社会及管治报告,1653580800000,2022,finalpage/2022-05-27/1213517593.PDF,2023-01-17,246,967884
9,000513,丽珠集团,gssz0000513,1205053842,丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2017年度环境、社会及管治报告,1528732800000,2018,finalpage/2018-06-12/1205053842.PDF,2023-01-17,70,36580


In [262]:
df.query("secCode == '000513'")

,secCode,secName,orgId,announcementId,announcementTitle,announcement_title,announcementTime,announcement_year,adjunctUrl,record_time,pages_num,words_num
7,000513,丽珠集团,gssz0000513,1210132133,丽珠集团：2020年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2020年度环境、社会及管治报告,1622476800000,2021,finalpage/2021-06-01/1210132133.PDF,2023-01-17,138,75402
8,000513,丽珠集团,gssz0000513,1213517593,丽珠集团：2021年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2021年度环境、社会及管治报告,1653580800000,2022,finalpage/2022-05-27/1213517593.PDF,2023-01-17,246,967884
9,000513,丽珠集团,gssz0000513,1205053842,丽珠集团：2017年度环境、<em>社会</em><em>及</em><em>管治</em>报告,丽珠集团：2017年度环境、社会及管治报告,1528732800000,2018,finalpage/2018-06-12/1205053842.PDF,2023-01-17,70,36580


In [250]:
df2 = df.groupby(['secCode','announcement_year'])['announcementTitle'].count()
df2

secCode  announcement_year
000039   2017                 1
         2018                 1
         2019                 1
         2020                 1
000166   2019                 1
000507   2021                 1
         2022                 1
000513   2018                 1
         2021                 1
         2022                 1
Name: announcementTitle, dtype: int64

In [255]:
df2.to_csv('../data/SearchRes2.csv')

#### ../data/SearchRes2.csv

In [259]:
df3 = pd.read_csv('../data/SearchRes2.csv',dtype=str)
index = df3['secCode'] == '000039'
df3[index]

,secCode,announcement_year,announcementTitle
0,000039,2017,1
1,000039,2018,1
2,000039,2019,1
3,000039,2020,1


In [140]:
df.groupby(['secCode','announcement_year']).value_counts()


secCode  announcement_year  secName  orgId        announcementTitle                                                  announcementTime  adjunctUrl                           record_time
39       2017               中集集团     gssz0000039  中集集团：2016年环境、<em>社会</em><em>及</em><em>管治</em>报告                    1498838400000     finalpage/2017-07-01/1203671603.PDF  2023-01-17     1
         2018               中集集团     gssz0000039  中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告      1522166400000     finalpage/2018-03-28/1204526484.PDF  2023-01-17     1
         2019               中集集团     gssz0000039  中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告       1553702400000     finalpage/2019-03-28/1205949450.PDF  2023-01-17     1
         2020               中集集团     gssz0000039  中集集团：<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告（2019）     1585238400000     finalpage/2020-03-27/1207411179.PDF  2023-01-17     1
166      2019               申万宏源     qsgn0000301  申万宏源：2018年

In [139]:
df.groupby(['secCode'])['announcement_year'].count()


secCode
39        4
166       1
507       2
513       6
585       1
661       1
723       1
786       1
860       1
877       1
921       5
961       2
987       1
2128      1
2180      2
2236      1
2242      4
2410      2
2415      4
2506      2
2672      2
2703      2
2910      4
2929      1
3816      1
300012    2
300070    2
300086    1
300181    1
300185    1
300682    2
300759    2
300948    1
300957    1
301039    1
301062    2
600011    6
600012    4
600016    4
600096    1
600101    1
600176    1
600188    1
600196    1
600219    2
600276    1
600350    1
600362    4
600461    1
600497    1
600548    1
600585    1
Name: announcement_year, dtype: int64

#### ?

In [68]:
a = [i for i in json_res.values()]

SyntaxError: can't use starred expression here (2205420745.py, line 1)

In [109]:
param_dict

{'searchkey': '社会及管治',
 'sdate': '',
 'edate': '',
 'isfulltext': 'false',
 'sortName': 'stockcode_cat',
 'sortType': 'asc',
 'pageNum': '',
 'type': ''}

In [110]:
url = 'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey={searchkey}&sdate={sdate}&edate={edate}&isfulltext={isfulltext}&sortName={sortName}&sortType={sortType}&pageNum={pageNum}&type={type}'


In [111]:
url.format(**param_dict)

'http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=&type='

#### 判断该announcement是否值得保留

In [272]:
def rightOrNot(inte):
    res = {'a':3}
    if inte == 0:
        return {}
    return res

In [273]:
# 不打算保留：
res = [1,0,1,1,1,1,1]
res_list = [rightOrNot(r) for r in res]
res_list

[{'a': 3}, {}, {'a': 3}, {'a': 3}, {'a': 3}, {'a': 3}, {'a': 3}]

In [274]:
from reFormat import to_dataframe

In [275]:
to_dataframe(res_list)

,a
0,3.0
1,NaN
2,3.0
3,3.0
4,3.0
5,3.0
6,3.0


#### 测试生成器时间快append多少

In [303]:
param_dict = {
    # 搜索的关键词，如【'企业社会责任'】
    'searchkey':'社会及管治',
    # 发布起始时间，如【'2023-01-04'】【''表示不进行限制 】 
    'sdate':'', 
    # 发布截止时间，如【'2023-02-24'】【''表示不进行限制 】
    'edate':'', 
    # 是否进行全文搜索，【'false'为只搜索标题，'true'为搜索全文 】
    'isfulltext':'false', 
    # 按什么进行排序，【'pubdate'为按时间排序，'stockcode_cat'为按代码排序，'nothing'为按相关度排序 】
    'sortName':'stockcode_cat', 
    # 升序还是降序，【'asc'为升序，'desc'为降序 】
    'sortType':'asc', 
    # 页码，【如'1'】
    'pageNum':'1', 
    # 板块筛选，【''为全部板块，'shj'为深沪京，'s'为三板... 】
    'type':'shj', 
}
from search_main import *

In [301]:
def f(param_dict,q):
    print(param_dict)
    res = query_all_dicts(param_dict=param_dict)
    print(param_dict)
    q.put(res)
    return

In [304]:
import multiprocessing as mp
q1 = mp.Queue(1)
p1 = mp.Process(target=f,args=(param_dict,q1,))
p1.start()
res = q1.get()
p1.join()


{'searchkey': '社会及管治', 'sdate': '', 'edate': '', 'isfulltext': 'false', 'sortName': 'stockcode_cat', 'sortType': 'asc', 'pageNum': '1', 'type': 'shj'}
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=1&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=2&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=3&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=4&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/fulltextSearch/full?searchkey=社会及管治&sdate=&edate=&isfulltext=false&sortName=stockcode_cat&sortType=asc&pageNum=5&type=shj 】
将要POST：【 http://www.cninfo.com.cn/new/f

In [342]:
# dicts = query_all_dicts(param_dict=param_dict)
dicts = res
dicts
# 格式化搜索结果——将存入csv的格式


[{'id': None,
  'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'announcementId': '1204526484',
  'announcementTitle': '中集集团：2017年<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcementTime': 1522166400000,
  'adjunctUrl': 'finalpage/2018-03-28/1204526484.PDF',
  'adjunctSize': 2190,
  'adjunctType': 'PDF',
  'storageTime': None,
  'columnId': '09020202||250101||251302',
  'pageColumn': 'SZZB',
  'announcementType': '01010503||010112||01239999',
  'associateAnnouncement': None,
  'important': None,
  'batchNum': None,
  'announcementContent': None,
  'orgName': None,
  'announcementTypeName': None},
 {'id': None,
  'secCode': '000039',
  'secName': '中集集团',
  'orgId': 'gssz0000039',
  'announcementId': '1205949450',
  'announcementTitle': '中集集团：2018<em>社会</em>责任暨环境、<em>社会</em><em>及</em><em>管治</em>报告',
  'announcementTime': 1553702400000,
  'adjunctUrl': 'finalpage/2019-03-28/1205949450.PDF',
  'adjunctSize': 21279,
  'adjunctType': 'PDF',
  'storageTi

In [315]:
print(len(dicts))
dicts = dicts*8
print(len(dicts))

228
1824


In [350]:
def maintain_or_not(announcement_title):
    reject_list = [
        '英文版',
        ]
    for reject in reject_list:
        if reject in announcement_title:
            return False
    return True


# if 1:
res_dicts = []
for announcement in dicts:
    res_dicts.append(announcement)
    # res_dicts += announcement

In [351]:
res_dicts = [reformat_dict(announcement) for announcement in dicts]
res_dicts_2 = [res_dict for res_dict in res_dicts if not maintain_or_not(res_dict['announcement_title'])]
print(len(res_dicts_2))

中集集团：2017年社会责任暨环境、社会及管治报告
中集集团：2018社会责任暨环境、社会及管治报告
中集集团：2016年环境、社会及管治报告
中集集团：社会责任暨环境、社会及管治报告（2019）
申万宏源：2018年度环境、社会及管治报告
珠海港：珠海港股份有限公司2020年度环境、社会及管治（ESG）报告
珠海港：2021年度环境、社会及管治（ESG）报告
丽珠集团：2020年度环境、社会及管治报告
丽珠集团：2021年度环境、社会及管治报告
丽珠集团：2017年度环境、社会及管治报告
丽珠集团：2016年度环境、社会及管治报告
丽珠集团：2018年度环境、社会及管治报告
丽珠集团：2019年环境、社会及管治报告
*ST东电：2018环境、社会及管治报告
长春高新：2021年度环境、社会及管治（ESG）报告
美锦能源：2021年环境、社会及管治报告
北新建材：2021年度环境、社会及管治（ESG）暨社会责任报告
顺鑫农业：2021年度环境、社会及管治（ESG）报告
天山股份：新疆天山水泥股份有限公司2021年度环境、社会及管治（ESG）报告
海信家电：2018年度环境、社会及管治报告
海信家电：2020年度环境、社会及管治报告
海信家电：2021年度环境、社会及管治报告
海信家电：2019年度环境、社会及管治报告
海信科龙：2017年度环境、社会及管治报告
中南建设：2021年环境、社会及管治报告（更新）
中南建设：关于2021年环境、社会及管治报告的更正公告
越秀金控：2021年环境、社会及管治（ESG）报告
电投能源：2021年度环境、社会及管治（ESG）报告
纳思达：2021环境、社会及管治报告(英文版)
纳思达：2021环境、社会及管治报告(中文版)
大华股份：2021年大华股份环境、社会及管治报告
九阳股份：2021年环境、社会及管治报告（英文版）
九阳股份：2020年环境、社会及管治报告
九阳股份：2020年环境、社会及管治报告（英文版）
九阳股份：2021年环境、社会及管治报告
广联达：2021年环境、社会及管治（ESG）报告
广联达：2021年环境、社会及管治（ESG）报告(英文版)
海康威视：2021年环境、社会及管治报告
海康威视：2020年环境、社会及管治报告（英文版）
海康威视：2021年环境、社会及管治报告（英文版）
海康威视：2020

In [352]:
[res_dict['announcement_title'] for res_dict in res_dicts_2]

['纳思达：2021环境、社会及管治报告(英文版)',
 '九阳股份：2021年环境、社会及管治报告（英文版）',
 '九阳股份：2020年环境、社会及管治报告（英文版）',
 '广联达：2021年环境、社会及管治（ESG）报告(英文版)',
 '海康威视：2020年环境、社会及管治报告（英文版）',
 '海康威视：2021年环境、社会及管治报告（英文版）',
 '庄园牧场：H股公告-2020年度环境、社会及管治报告（英文版）',
 '庄园牧场：H股公告-2021年度环境、社会及管治报告（英文版）',
 '朗新科技：2021年度环境、社会及管治（ESG）报告（英文版）',
 '南山铝业：山东南山铝业股份有限公司2021年度环境、社会及管治报告（英文版）',
 '新奥股份：新奥天然气股份有限公司2020年度环境、社会及管治报告（英文版）',
 '中国电建：中国电力建设股份有限公司2021年环境、社会及管治报告（英文版）',
 '中国中免：中国旅游集团中免股份有限公司2021年度环境、社会及管治报告（英文版)',
 '公牛集团：公牛集团2021年度环境、社会及管治报告（英文版）',
 '中谷物流：2021年环境、社会及管治报告（英文版）',
 '国联股份：2021年度环境、社会及管治报告(英文版)',
 '晨光股份：上海晨光文具股份有限公司2021年环境、社会及管治报告（英文版）',
 '传音控股：2021年度环境、社会及管治（ESG）报告【英文版】',
 '金山办公：金山办公2021年度环境、社会及管治（ESG）报告（英文版）']

In [ ]:


# 将结果转化为csv
df = to_dataframe(res_dicts)

# 生成所有announcement的pdf文件的名称和下载列表，将用于协程下载
pdf_urls = [(dict0['announcement_title']+'.pdf','http://static.cninfo.com.cn/'+dict0['adjunctUrl']) for dict0 in res_dicts]
# 使用协程下载pdf文件，并返回所有文件下载到的路径（类型：tuple）
dirs = async_downloads(pdf_urls,'./data/search_res')

# 使用多进程分析pdf列表，返回结果字典列表（每个字典对应一个pdf的分析结果）
parse_res = multi_process(parse_pdf,dirs)
pages_nums = [res['pages_num'] for res in parse_res]
words_nums = [res['words_num'] for res in parse_res]
df['pages_num'] = pages_nums
df['words_num'] = words_nums
# 将csv存到本地（可选）
df.to_csv('./res/announcements.csv',index=False,)

# 统计并存到本地
df2 = df.groupby(['secCode','announcement_year'])['announcementTitle'].count()
df2.to_csv('./res/times_per_code_per_year.csv')